In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import scipy.stats as sstat
from mpmath import mp
import math
import random

psd = pd.DataFrame
from tensorflow_probability.substrates import numpy as tfp
tfd = tfp.distributions

In [2]:
def single_lik(mu):
   y= tfd.MultivariateNormalFullCovariance(mu,sigma_obs)
   return y

In [ ]:
## mu_obs=np.array([1,4])
## mu1=[1,4]
## sigma_obs=np.array([[1,0.5],[0.5,1]])
## sigma1=[[1, 0.5],[0.5,1]]
## y_obs=tfd.MultivariateNormalFullCovariance(mu_obs,sigma_obs).sample(1000)
## y_obs

In [3]:
def prior():
    return tfd.MultivariateNormalFullCovariance(mu0,sigma0)

In [4]:
def proposal(theta1):
  return tfd.MultivariateNormalFullCovariance(theta,cov)

In [ ]:
## def my_kernel(u):
## return 1/(np.sqrt(2*math.pi))*np.exp(-1/2*u**2)

In [ ]:
## u=[1,2]
## my_kernel(u)

In [12]:
from numpy.linalg.linalg import transpose
def K_h(u,H):
    return 1/(np.sqrt(2*math.pi))*pow(np.linalg.det(H), -1/2)*np.exp(-1/2*transpose(u)*pow(H,-1)*u)

In [14]:
u= np.array([1,1])
H= np.array([[1,0],[0,1]])
K_h(u,H)

AxisError: ignored

In [ ]:
def K_h(u,h):
  return my_kernel(u/h)/h

In [ ]:
def S1(y):
  quantiles=[]
  for i in range(9):
      quantiles.append(np.quantile(y,0.1+0.1*i))         #quantili 0,1 0,2 0,3 0,4 0,5 0,6 0,7 0,8 0,9
  return quantiles

In [ ]:
def S2(y):
  return np.mean(y)

## sicuramente sbagliato

In [ ]:
S2(y_obs)

In [ ]:
def abc_algo (N, S, y_obs, h):
    s_obs = S(y_obs)
    theta=[]
    s = []
    y = []
    theta_0 = prior().sample()
    y_0 = single_lik(theta_0).sample() 
    s_0 = S(y_0)
    
    while K_h(np.linalg.norm(np.array(s_0)-np.array(s_obs)),h)<=0:
        theta_0 = prior().sample()
        y_0 = single_lik(theta_0).sample() 
        s_0 = S(y_0)
      
    theta.append(theta_0)
    s.append(s_0)
    y.append(y_0)
    theta_old = theta_0
    s_old = s_0
    y_old = y_0

    for i in range(N):
        theta_current = proposal(theta_old).sample()
        y_current = single_lik(theta_current).sample(1000)
        #y.append(y_current)
        s_current = S(y_current)
               
        prob = min([1.,(K_h(np.linalg.norm(np.array(s_current) - np.array(s_obs)),h)*prior().prob(theta_current)*proposal(theta_old).prob(theta_current))/(
             K_h(np.linalg.norm(np.array(s_old)-np.array(s_obs)),h)*prior().prob(theta_old)*proposal(theta_current).prob(theta_old))])
        u = tfd.Uniform(0,1).sample()
        if u < prob:
              theta_i = theta_current
              s_i = s_current
        else: 
              theta_i = theta_old
              s_i = s_old
              #y.pop()
              #y.append(y_old)

        theta_old = theta_i
        s_old = s_i
        
        theta.append(theta_i)
        s.append(s_i)
        
    return  theta

In [ ]:
##VARIABILI:
mu0 = np.array ([[3,6]])
sigma0 = np.array([[3,0.7],[0.7,3]])
n = 1000
mu_obs = np.array ([[1,4]])
sigma_obs=np.array([[1,0.5],[0.5,1]])
y_obs=tfd.MultivariateNormalFullCovariance(mu_obs,sigma_obs).sample(1000)

In [ ]:
prova = abc_algo (n, S2, y_obs, 0.2) 

LinAlgError: ignored